In [1]:
from train import main
import numpy as np
from PIL import Image
import cv2
import torch
from utils import get_loaders
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from model_V1 import UNET
import glob

In [12]:
def ordering_path(path):
    core_id = int(path.split('//')[-1][0])*10000 + 1000
    img_id = path.split('//')[-1]
    img_id = int(img_id[img_id.find('(')+len('('):img_id.rfind(')')])
    return img_id+core_id


In [2]:
#loading the model shape and weights 
model = UNET().to('cpu')
model.load_state_dict(torch.load('my_checkpoint_final.pth.tar')["state_dict"])

<All keys matched successfully>

In [3]:
#reading images and giving order to the path reader
TRAIN_IMG_DIR = "D /"
TRAIN_MASK_DIR = "D:// t//"
VAL_IMG_DIR = "D:// /"
VAL_MASK_DIR = "D:// /"


TRAIN_IMG_list = glob.glob(TRAIN_IMG_DIR+'*.png')

TRAIN_MASK_list = glob.glob(TRAIN_MASK_DIR+'*.png')

VAL_IMG_list = glob.glob(VAL_IMG_DIR+'*.png')

VAL_MASK_list = glob.glob(VAL_MASK_DIR+'*.png')



In [5]:
def get_prediction(path, model):
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image = image.reshape(1,1, image.shape[0],image.shape[1])
    image = torch.from_numpy(image)
    pred = torch.sigmoid(model(image))
    pred = pred.detach().numpy()
    pred = pred[0,0,:,:]
    return pred



In [11]:
# Get prediction for test set
for i, path in enumerate (VAL_IMG_list[:]):
    pred = get_prediction(path, model)
    cv2.imwrite( 'D://pred_test//' + path.split('\\')[-1],pred)
    print (round(i*100/len(VAL_IMG_list)), end="\r")

In [13]:
# Get prediction for train set
for i, path in enumerate (TRAIN_IMG_list[:]):
    pred = get_prediction(path, model)
    cv2.imwrite( 'D://pred_train//' + path.split('\\')[-1],pred)
    print (round(i*100/len(TRAIN_IMG_list)), end="\r")